In [1]:
import sys
import pprint
import datetime
from tlslite.utils import keyfactory
import pymongo
import pprint
import math
import pandas as pd
import numpy as np
from plotly import tools
import plotly.graph_objs as go
import plotly.offline as py
import os
import requests
import time
from bs4 import BeautifulSoup

py.init_notebook_mode(connected=True)

In [2]:
client = pymongo.MongoClient('18.8.8.120')
cydb = client.cy_useract
issues = pd.DataFrame(list(cydb.useract_info.find()))
del issues['_id']
len(issues)
valid_data = issues.loc[issues['IMEI'].apply(lambda x:len(x.split(';')) == 2)]

client = pymongo.MongoClient('18.8.8.209')
cydb = client.cy
#topapps = pd.DataFrame(list(cydb.topapps.find(projection={'name': 1, 'package_name':1})))
#del topapps['_id']
#cy_packages = pd.DataFrame(list(cydb.cy_packages.find()))
#del cy_packages['_id']

In [3]:
len(issues)

585645

## 获取包名及函数定义

In [ ]:
#获取包对应的应用名
total_apps = pd.DataFrame(list(cydb.package2name.find(projection={'_id':0, 'name': 1, 'package_name':1})))
total_apps_names = total_apps.set_index('package_name').to_dict()['name']

In [ ]:
###获取包名
def get_name_from_google(package_name):
    name = 'Not Found'
    url = "https://play.google.com/store/apps/details"
    querystring = {"id":package_name}

    headers = {
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        'accept-encoding': "gzip, deflate, br",
        'accept-language': "zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7",
        'upgrade-insecure-requests': "1",
        'cookie': 'SID=DAaVKqDYVfUDOvDttkKw4L9_fFoj3k4amZyPFCJv2-JrhZoD2gLvVPLqDS9ipsTYjwXWmw.; HSID=Az021_6ePZogKY8cM; SSID=ACpzbNvy8_x8Vnybk; APISID=7S1tb3ZsgGv_bYvR/ACjOTyZPnt4umXRCq; SAPISID=xjIFc_YmAa63ey27/AmEbiR4E6aujvkXPS; NID=130=ab4LxWGm30bUFH_KitKmxCkBbExye2K9qU5VTdygb49TBNA1a8pN4jyad6LYrAPvhIpVO6_iNDLTOJuWxrwzgPd3qCm6uC6LFx8txajCZp-xDO7tdQIGf_eQYLM_BVGPdAQ47Oa2WUX_9DshsR3iJItArmKVShCwPvA2HCFO6XLBDaHjyRpXAhPrYKZfGIQ0PVYRkT8hBjzDCtPM4VcHKbSmMi0SCQ1dVGJDTKbALIYYCBwODH_mQsStZA4FHzSzT71NG-xMPONgrzmi9h6Nm1Xo; OGPC=19005936-1:; S=billing-ui-v3=H2iK8QW8I6T65a977LlFh58Un3NygpND:billing-ui-v3-efe=H2iK8QW8I6T65a977LlFh58Un3NygpND; SIDCC=AEfoLebYzi5VJ4REqjhIorMwY7WG7kadx86R29vcphaujmkiGT8I2tDqnP5_VwSYG_kLoRECJg; 1P_JAR=2018-5-18-7',
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36  \
        (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        'x-client-data': "CKe1yQEIibbJAQiltskBCMG2yQEIq5jKAQipncoBCN6eygEIqKPKARiSo8oB",
        'cache-control': "no-cache",
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    if response.status_code == 200:
        soup_app = BeautifulSoup(response.text, 'lxml')
        href_all = soup_app.find_all('h1')
        name = href_all[0].span.string
        
    return name

In [ ]:
def get_bar_table_data(cur_month, last_month, rate=True, need_other=True):
    top30_click = cur_month[-30:]
    others = cur_month.sum() - top30_click.sum()
    
    if need_other:
        top31_click = top30_click.append(pd.Series({'Others':others})).sort_values()
    else:
        top31_click = top30_click
    appclick_diff = cur_month.sub(last_month, fill_value=0)[list(top30_click.index)]

    package_name_diff = list(appclick_diff.index)
    package_name_diff.reverse()

    app_name_diff = []
    for package in package_name_diff:
        if package in total_apps_names.keys():
            if total_apps_names[package] != 'Not Found':
                app_name_diff.append(total_apps_names[package])
            else:
                app_name_diff.append(package)
        else:
            print('1 get package name from google:' + package)
            name = get_name_from_google(package)
            total_apps_names[package] = name
            if name == 'Not Found':
                app_name_diff.append(package)
            else:
                app_name_diff.append(name)
                cydb.package2name.insert_one({'name':name, 'package_name':package})
            time.sleep(2)
    
    value_diff = list(appclick_diff.values)
    value_diff.reverse()
    
    packages_name = []
    package_list = list(total_apps['package_name'])
    for package in top31_click.index:
        if package in total_apps_names.keys():
            if total_apps_names[package] != 'Not Found':
                packages_name.append(total_apps_names[package])
            else:
                packages_name.append(package)
        else:
            print('3 get package name from google:' + package)
            name = get_name_from_google(package)
            total_apps_names[package] = name
            if name == 'Not Found':
                packages_name.append(package)
            else:
                packages_name.append(name)
                cydb.package2name.insert_one({'name':name, 'package_name':package})
            time.sleep(2)
    
    bar_data = {}
    bar_data['y'] = packages_name
    bar_data['x'] = top31_click.values
    if rate:
        bar_data['text'] = top31_click.apply(lambda x:'{:.2f}%'.format(x*100)).values
        table_data = [app_name_diff, ['{:.2f}%'.format(x*100) for x in value_diff]]
    else:
        bar_data['text'] = top31_click.apply(lambda x:round(x,1)).values
        table_data = [app_name_diff, [round(x,1) for x in value_diff]]
    
    
    return bar_data, table_data

In [ ]:
def draw_bar_table(bar_data, table_data, height, width, title, table_pos={'x':[0.45, 0.9], 'y':[0, 0.8]}):
    table_trace = go.Table(
        domain=table_pos,
        columnwidth = [52]*2,
        columnorder=[0, 1],
        header = dict(height = 25,
                    values = [['<b>名称</b>'],['<b>与上月同比</b>'],],
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=14),
                    fill = dict(color='#2d5b6c')),
        cells = dict(values = table_data,
                    line = dict(color='#000000'),
                    align = ['left'] * 2,
                    font = dict(color=['#FFFFFF'] * 2, size=12),
                    #format = [None]*4 + [',.2f'],
                    #prefix = [None] * 2 + ['$', u'\u20BF'],
                    #suffix=[None] * 4,
                    height = 25,
                    fill = dict(color=['#68a4b9', '#526372']))
    )

    trace = go.Bar(y=bar_data['y'], x=bar_data['x'], orientation = 'h',
                  text = bar_data['text'],textposition = 'outside',)

    layout = go.Layout(
        title=title,
        height=height,
        width=width,
        margin=dict(l=300,r=0),
        yaxis = dict(ticklen=2)
    )

    fig = go.Figure(data=[trace,table_trace],layout=layout)
    py.iplot(fig)


## 获取上两月，上月及当月的数据

In [ ]:

last_2month = datetime.datetime(2018, 4, 18).timestamp()
last_month = datetime.datetime(2018, 5, 18).timestamp()
cur_month = datetime.datetime(2018, 6, 18).timestamp()

df_last_month = valid_data[(valid_data['wallTime'] >= last_2month) & (valid_data['wallTime'] < last_month)]
df_cur_month = valid_data[(valid_data['wallTime'] >= last_month) & (valid_data['wallTime'] < cur_month)]

## 点击排行

In [ ]:
imeis_cur_month = len(df_cur_month['IMEI'].unique())
imeis_last_month = len(df_last_month['IMEI'].unique())

total_appclick_cur_month = df_cur_month[df_cur_month['name'] == 'appclick']
total_appclick_last_month = df_last_month[df_last_month['name'] == 'appclick']

In [ ]:
appclick_cur_month = total_appclick_cur_month.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick_cur_month))
appclick_last_month = total_appclick_last_month.groupby(['attrs_packagename']).size().sort_values().truediv(len(total_appclick_last_month))

title='点击次数排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick_cur_month), imeis_cur_month)
bar_data, table_data = get_bar_table_data(appclick_cur_month, appclick_last_month)

In [ ]:
draw_bar_table(bar_data, table_data, height=1300, width=1300, title=title)

In [ ]:
click_by_phone_cur_month = total_appclick_cur_month.groupby(['attrs_packagename']).size().sort_values().truediv(imeis_cur_month)
click_by_phone_cur_month = click_by_phone_cur_month.apply(lambda x: round(x,1))
click_by_phone_last_month = total_appclick_last_month.groupby(['attrs_packagename']).size().sort_values().truediv(imeis_last_month)
click_by_phone_last_month = click_by_phone_last_month.apply(lambda x: round(x,1))

title='人均点击排名<br>总点击数:{}  设备数:{}'.format(len(total_appclick_cur_month), imeis_cur_month)
bar_data, table_data = get_bar_table_data(click_by_phone_cur_month, click_by_phone_last_month, False)
draw_bar_table(bar_data, table_data, height=1300, width=1300, title=title)

## 安装卸载排行

In [ ]:
def install_rate(cur_month, last_month, label,title,height=1200,width=1200,by_imei=True):
    data_cur = cur_month[cur_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    data_last = last_month[last_month.name == label].groupby(['IMEI', 'attrs_packagename']).size()
    
    package_info_cur = data_cur.reset_index()
    package_info_last = data_last.reset_index()
    
    imeis_cur = len(cur_month['IMEI'].unique())
    imeis_last = len(last_month['IMEI'].unique())
    total_cur = package_info_cur.groupby('attrs_packagename').size().sort_values().sum()
    total_last = package_info_last.groupby('attrs_packagename').size().sort_values().sum()
    
    if by_imei:
        divisor_cur = imeis_cur
        divisor_last = imeis_last
    else:
        divisor_cur = total_cur
        divisor_last = total_last
        
    package_rate_cur = package_info_cur.groupby('attrs_packagename').size().sort_values()/divisor_cur
    package_rate_last = package_info_last.groupby('attrs_packagename').size().sort_values()/divisor_last
    
    bar_data, table_data = get_bar_table_data(package_rate_cur, package_rate_last, rate=True, need_other=False)
    title= title + '<br>总次数:{}  总设备数:{}'.format(str(int(total_cur)), imeis_cur)
 
    draw_bar_table(bar_data, table_data, height, width, title)

In [ ]:
install_rate(df_cur_month, df_last_month, 'appinstall', '安装应用排行', height=1300,width=1300,by_imei=False)

In [ ]:
install_rate(df_cur_month, df_last_month, 'appinstall', '安装应用覆盖率排行', height=1300,width=1300)

In [ ]:
install_rate(df_cur_month, df_last_month,'appremove', '卸载应用排行',height=1400,width=1300)

## Camera 应用 

In [ ]:
chinese_name_photos = {'photo':'普通拍照', 'panorama':'全景', 'facebeauty':'美颜', 'hdr':'逆光', 'nightvideo':'夜景',
                'selfiepanorama':'全景自拍', 'polaroid':'心情照片','gifshot':'GIF动画','profession':'高级', 'arcfilter':'滤镜'}
chinese_name_videos = {'video':'普通摄像','slowmotion':'慢动作摄影', 'timelapse':'延时摄影'}

In [ ]:
def draw_camera(cur, last, height, width, title, table_pos={'x':[0.45, 0.9], 'y':[0, 0.86]}):
    cur_rate = cur.apply(lambda x:x/cur.sum())
    last_rate = last.apply(lambda x:x/last.sum())
    diff_data = cur_rate.sub(last_rate, fill_value=0)
    table_data = [diff_data.index, ['{:.2f}%'.format(x*100) for x in diff_data]]
    bar_data = {'y':cur_rate.index, 'x':cur_rate.values,
                'text':cur_rate.apply(lambda x:'{:.1f}%'.format(x*100)).values}
    draw_bar_table(bar_data, table_data, height, width, title, table_pos)

In [ ]:
def get_camera_data(df):
    takephoto_data_cur = df[df['name'] == 'TakePhoto']
    video_data_cur = df[df['name'] == 'Video']

    total_camera_cur = len(takephoto_data_cur)
    total_video_cur = len(video_data_cur)
    total_used_cur = (len(takephoto_data_cur) + len(video_data_cur))

    capturemode_cur = takephoto_data_cur.groupby(['attrs_Capturemode']).size().sort_values()

    capture_dict_cur = {}
    total_camera_dict_cur = {}
    for p in chinese_name_photos:
        if p in capturemode_cur.index:
            capture_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
            total_camera_dict_cur[chinese_name_photos[p]] = capturemode_cur[p]
        else:
            total_camera_dict_cur[chinese_name_photos[p]] = 0
            capture_dict_cur[chinese_name_photos[p]] = 0

    capture_chinese_data = pd.Series(capture_dict_cur).sort_values()

    video_mode = video_data_cur.groupby(['attrs_Videomode']).size().sort_values()
    video_dict_cur = {}
    for p in chinese_name_videos:
        if p in video_mode.index:
            video_dict_cur[chinese_name_videos[p]] = video_mode[p]
            total_camera_dict_cur[chinese_name_videos[p]] = video_mode[p]
        else:
            capture_dict_cur[chinese_name_videos[p]] = 0
            total_camera_dict_cur[chinese_name_videos[p]] = 0

    video_chinese_data = pd.Series(video_dict_cur).sort_values()
    total_chinese_data = pd.Series(total_camera_dict_cur).sort_values()
    
    return [capture_chinese_data,video_chinese_data,total_chinese_data]

In [ ]:
cur_camera = get_camera_data(df_cur_month)
last_camera = get_camera_data(df_last_month)

In [ ]:
draw_camera(cur_camera[0], last_camera[0], 550, 1200, '拍照使用排名<br>总使用次数:{}'.format(cur_camera[0].sum()))

In [ ]:
draw_camera(cur_camera[1], last_camera[1], 400, 1200, '录像使用排名<br>总使用次数:{}'.format(cur_camera[1].sum()),
           table_pos={'x':[0.45, 0.9], 'y':[0, 0.6]})

In [ ]:
draw_camera(cur_camera[2], last_camera[2], 600, 1200, '相机使用排名<br>总使用次数:{}'.format(cur_camera[2].sum()))

### 销量统计

In [ ]:
#valid_data.groupby(['IMEI', 'wallTime']).size()

In [125]:
imeis = len(valid_data['IMEI'].unique())

In [127]:
imeis

699

In [ ]:
d518 = datetime.datetime(2018, 1, 1).timestamp()

In [ ]:
vvdata = valid_data[valid_data['registerTime'] < d518]
#imeis = len(vvdata['IMEI'].unique())
#vvdata = valid_data
register = vvdata.groupby(['IMEI', 'registerTime']).size()
register_data = {}

for imei, group in register.reset_index().groupby(['IMEI']):
    register_date = datetime.datetime.fromtimestamp(group.iloc[-1]['registerTime'])
    register_data[imei] = register_date

In [ ]:
psd = pd.DataFrame.from_dict(register_data, orient='index')

In [ ]:
psd

In [ ]:
register_count = psd.groupby([0]).size()

In [ ]:
cc = register_count.resample('D').sum().fillna(0)

In [ ]:
paper_bgcolor='#242640'
plot_bgcolor='#242640'
paper_bgcolor='#FFFFFF'
plot_bgcolor='#FFFFFF'
text_color = '#000000'
x_color = '#000000'
y_color = '#000000'
title_color = '#000000'
legend_color = '#000000'

In [ ]:
cc.index.strftime('%m-%d')

In [ ]:
cc.index.strftime('%y-%m-%d')

In [ ]:
cc.values

In [ ]:
bandxaxis = go.XAxis(
        type='date',
        #range=[(created_by_day.index[start] - 1).strftime('%y-%m-%d'),
        #       (created_by_day.index[-1] + 1).strftime('%y-%m-%d'),],
        ticks="outside", 
        showticklabels=True,
        ticktext=list(cc.index.strftime('%m-%d')),
        tickvals=list(cc.index.strftime('%y-%m-%d')),
        tickfont=dict(size=11),
        #tickfont=dict(size=11,color='#B9B9C3'),
        ticklen=10,showgrid=True,
        #color='#FFFFFF'
    )

In [ ]:
data=[go.Bar(
        x = cc.index,
        y = cc.values,
        text=cc.values, 
        textposition='outside',)
        #marker=dict(color='#7D858C')), 
]


In [ ]:
layout = go.Layout(
    title='销量趋势图<br>总数:' + str(imeis),
    titlefont=dict(color=title_color,size=25),
    height=1200,
    width=2000,
    bargap=0.1,
    xaxis = bandxaxis,
    yaxis = dict(ticklen=10),
    #bargroupgap=0.05,
    #paper_bgcolor=paper_bgcolor,
    #plot_bgcolor=plot_bgcolor,
    legend=dict(
        x=0.1,
        font=dict(size=16),
    )
)

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

## OTA 统计

### ota 包升级率

In [110]:
def draw_stats_table(table_data, height, width, title, annotation):
    table_trace = go.Table(
        columnwidth = [52]*3,
        columnorder=[0, 1, 2],
        header = dict(height = 25,
                    values = [['<b>版本</b>'],['<b>升级率</b>'],['<b>升级个数</b>']],
                    line = dict(color='#000000'),
                    align = ['left'] * 3,
                    font = dict(color=['#FFFFFF'] * 3, size=14),
                    fill = dict(color='#2d5b6c')),
        cells = dict(values = table_data,
                    line = dict(color='#000000'),
                    align = ['left'] * 3,
                    font = dict(color=['#FFFFFF'] * 3, size=12),
                    #format = [None]*4 + [',.2f'],
                    #prefix = [None] * 2 + ['$', u'\u20BF'],
                    #suffix=[None] * 4,
                    height = 25,
                    fill = dict(color=['#68a4b9', '#526372']))
    )
    
    annotations = { "text": annotation,
               "showarrow": False,
               "xref": "paper",
               "yref": "paper",
               "x": 0,
               "y": 0}
    
    layout = go.Layout(
        title=title,
        height=height,
        width=width,
        annotations=[annotations],
        #margin=dict(l=300,r=0),
        #yaxis = dict(ticklen=2)
    )

    fig = go.Figure(data=[table_trace],layout=layout)
    py.iplot(fig)


In [79]:
device_models = valid_data.deviceModel.unique()

In [138]:
d1 = valid_data[valid_data['deviceModel'] == device_models[1]]['IMEI'].unique()

In [137]:
d2 = valid_data[valid_data['deviceModel'] == device_models[0]]['IMEI'].unique()

In [139]:
for d in d2:
    if d in d1:
        print(d)

354147042107087;354147043107086
354147020876752;354147020138542
123456789012347;123456789012347


In [134]:
valid_data['IMEI'].unique()

array(['354147042070715;354147042085713',
       '354147042070251;354147042085259',
       '354147042334574;354147042384579',
       '354147042054644;354147042054677',
       '784354325132123;458456143251321',
       '444444444444446;354147040127103',
       '354147042334145;354147042384140',
       '354147043175851;354147042358698',
       '354147042334152;354147042384157',
       '354147042334129;354147042384124',
       '354147042054560;354147089746516',
       '354288080016275;354288080016283',
       '354147095475217;354147062985313',
       '354147075984618;354147016987522',
       '354147042089541;354147042013269',
       '354147042334749;354147042384744',
       '868955030027250;868955030027268',
       '868954030020431;868954030020449',
       '868954030026818;868954030026826',
       '354147042071317;354147042086315',
       '868954030020878;868954030020886',
       '868954030021256;868954030021264',
       '868954030026016;868954030026024',
       '868954030021090;8689540300

In [122]:
def get_ota_stats(device_model, height, width):
    d1 = valid_data[(valid_data['deviceModel'] == device_model)]
    device_sum = len(d1['IMEI'].unique())
    d1_upgrade = valid_data[(valid_data['deviceModel'] == device_model) & (valid_data['name'] == 'FinishUpgrade')]

    d1_upgrade['from'] = d1_upgrade['attrs_UpgradeVersionInfo'].apply(lambda x:x.split('->')).apply(
                    lambda x:x[0].split('_')[2])
    d1_upgrade['to'] = d1_upgrade['attrs_UpgradeVersionInfo'].apply(lambda x:x.split('->')).apply(
                    lambda x:x[1].split('_')[2])

    d1_upgrade_rate = d1_upgrade.groupby(['to']).count()['IMEI'].apply(lambda x:'{}%'.format(round(x*100/device_sum,2))).values
    d1_upgrade_num = d1_upgrade.groupby(['to']).count()['IMEI'].values
    table_data = [d1_upgrade.groupby(['to']).count()['IMEI'].index, d1_upgrade_rate, d1_upgrade_num]
    title = '{} OTA包升级率<br>总设备数:{}'.format(device_model, device_sum)
    row = len(d1_upgrade.groupby(['to']).count()['IMEI'].index) + 11
    #下载成功率
    d1_failed = len(valid_data[(valid_data['deviceModel'] == device_model) & (valid_data['name'] == 'FailDownload')])
    d1_success = len(valid_data[(valid_data['deviceModel'] == device_model) & (valid_data['name'] == 'FinishDownload')])
    download_rate = round(d1_success*100/(d1_success+d1_failed),2)
    
    annotations = '下载成功率:{}%  失败次数:{} 成功次数:{}'.format(download_rate, d1_failed, d1_success)
    draw_stats_table(table_data, row*25, width, title, annotations)

In [123]:
get_ota_stats(device_models[0],600,800)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [124]:
get_ota_stats(device_models[1],600,800)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### ota 包下载成功率

In [91]:
d1_success = valid_data[(valid_data['deviceModel'] == device_model[0]) & (valid_data['name'] == 'FinishDownload')]

In [89]:
d1_success

,IMEI,androidId,appKey,appVersion,attrs_Capturemode,attrs_DownloadFailReason,attrs_NetType,attrs_UpgradeVersionInfo,attrs_Videomode,attrs_applabel,...,deviceManufacturer,deviceModel,deviceOsVersion,deviceSdk,localeCountry,localeLanguage,name,registerTime,sequence,wallTime
204182,354147024836547;354147029836542,8bcabf332888f96e,1666CBB09EA6735B,8.0.0.aw,NaN,NaN,NaN,NaN,NaN,NaN,...,soda,soda S1,8.0.0,26,US,en,FinishDownload,1525412992,5,1525512760
243122,354147040016397;354147040016405,187e7a87626ba39d,1666CBB09EA6735B,8.0.0.aw,NaN,NaN,NaN,NaN,NaN,NaN,...,soda,soda S1,8.0.0,26,CN,zh,FinishDownload,1526034560,3,1526035306
